In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [6]:
import pandas as pd
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import AutoTokenizer

In [3]:
dataset_name = "Intel/orca_dpo_pairs"
DATA_PATH = f"Sharathhebbar24/{dataset_name.split('/')[1]}"

In [4]:
dataset_name, DATA_PATH

('Intel/orca_dpo_pairs', 'Sharathhebbar24/orca_dpo_pairs')

In [5]:
dataset = load_dataset(dataset_name, split="train")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['system', 'question', 'chosen', 'rejected'],
    num_rows: 12859
})

In [7]:
def clean_dataset(example):
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['question']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    # Format chosen answer
    chosen = example['chosen'] + "<|im_end|>\n"

    # Format rejected answer
    rejected = example['rejected'] + "<|im_end|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }


In [8]:
tokenizer = AutoTokenizer.from_pretrained("Sharathhebbar24/SSH_300M")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [9]:
original_columns = dataset.column_names
dataset = dataset.map(
    clean_dataset,
    remove_columns=original_columns
)

dataset

Map:   0%|          | 0/12859 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the GPT2TokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 12859
})

In [10]:
DATA_PATH

'Sharathhebbar24/orca_dpo_pairs'

In [11]:
dataset.push_to_hub(DATA_PATH)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Sharathhebbar24/orca_dpo_pairs/commit/d42171f7ffe32d91e54b4df46a4e81c177ffcfc1', commit_message='Upload dataset', commit_description='', oid='d42171f7ffe32d91e54b4df46a4e81c177ffcfc1', pr_url=None, pr_revision=None, pr_num=None)